In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import re
import xml.etree.ElementTree as ET
import xmltodict
import collections
import itertools
from ratelimit import limits, sleep_and_retry
from config import api_key

In [2]:
StateSelected = ['GA']

In [3]:
zipcodes=pd.read_csv('free-zipcode-database-Primary.csv')
zipcodes.head()

,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
0,705,STANDARD,AIBONITO,PR,PRIMARY,18.14,-66.26,NA-US-PR-AIBONITO,False,NaN,NaN,NaN
1,610,STANDARD,ANASCO,PR,PRIMARY,18.28,-67.14,NA-US-PR-ANASCO,False,NaN,NaN,NaN
2,611,PO BOX,ANGELES,PR,PRIMARY,18.28,-66.79,NA-US-PR-ANGELES,False,NaN,NaN,NaN
3,612,STANDARD,ARECIBO,PR,PRIMARY,18.45,-66.73,NA-US-PR-ARECIBO,False,NaN,NaN,NaN
4,601,STANDARD,ADJUNTAS,PR,PRIMARY,18.16,-66.72,NA-US-PR-ADJUNTAS,False,NaN,NaN,NaN


In [4]:
gazipcodes= zipcodes[zipcodes.State.isin(StateSelected)]

In [5]:
gazipcodes=gazipcodes.head()

## Perform API Calls

In [6]:

#conn = http.client.HTTPSConnection("search.onboard-apis.com") 

params = {'accept': "application/json",
          'apikey': "9d078487e223b1c4d54c3f3a3f628803"} 

In [7]:
FullAddress=[]
Street=[]
Latitude=[]
Longitude=[]
CityStateZip=[]
pd.Series(FullAddress)
pd.Series(Latitude)
pd.Series(Longitude)
Data_set=pd.DataFrame({'FullAddress':pd.Series(FullAddress),'Street':pd.Series(Street),'CityStateZip':pd.Series(CityStateZip),'Latitude':pd.Series(Latitude),'Longitude':pd.Series(Longitude)})

In [8]:
Data_set

,FullAddress,Street,CityStateZip,Latitude,Longitude


In [9]:
url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode=postalcode&page=1&pagesize=100"
for index, row in gazipcodes.iterrows():
    postalcode=row['Zipcode']
    url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode={postalcode}&page=1&pagesize=1000"
    res = requests.get(url,headers=params)
    data =json.loads(res.text)
    we=data
    try:
        d=pd.DataFrame(data['property'])
        d['FullAddress']=d['address'].apply(lambda a:a['oneLine'])
        d['Street']=d['address'].apply(lambda d:d['line1'])
        d['CityStateZip']=d['address'].apply(lambda d:d['line2'])
        d['Latitude']=d['location'].apply(lambda c:c['latitude'])
        d['Longitude']=d['location'].apply(lambda b:b['longitude'])
        datatoappend=d[['FullAddress','Street','CityStateZip','Latitude','Longitude']]
        Data_set=Data_set.append(datatoappend)
        
    
            
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    


In [10]:
Data_set.head()

,FullAddress,Street,CityStateZip,Latitude,Longitude
0,"109 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",109 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.417930,-84.937120
1,"137 TAYLOR BRIDGE RD SW # A, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW # A,"CALHOUN, GA 30701",34.418477,-84.937531
2,"137 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.418477,-84.937531
3,"314 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",314 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420551,-84.983000
4,"310 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",310 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420560,-84.983002


In [11]:
Data_set['Zestimate']=" "
Data_set['LastSoldPrice']=" "
Data_set['Year Built']=" "
Data_set['Nof of Bathrooms']=" "
Data_set['No of Bedrooms']=" "
Data_set.head()

,FullAddress,Street,CityStateZip,Latitude,Longitude,Zestimate,LastSoldPrice,Year Built,Nof of Bathrooms,No of Bedrooms
0,"109 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",109 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.417930,-84.937120,,,,,
1,"137 TAYLOR BRIDGE RD SW # A, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW # A,"CALHOUN, GA 30701",34.418477,-84.937531,,,,,
2,"137 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.418477,-84.937531,,,,,
3,"314 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",314 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420551,-84.983000,,,,,
4,"310 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",310 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420560,-84.983002,,,,,


In [12]:
Data_set['Zilliw format add']=Data_set['Street'].replace(" ", "+",regex=True)
Data_set['Zilliw format Zipcity']=Data_set['CityStateZip'].replace(" ", "+",regex=True)
Data_set.head()

,FullAddress,Street,CityStateZip,Latitude,Longitude,Zestimate,LastSoldPrice,Year Built,Nof of Bathrooms,No of Bedrooms,Zilliw format add,Zilliw format Zipcity
0,"109 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",109 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.417930,-84.937120,,,,,,109+TAYLOR+BRIDGE+RD+SW,"CALHOUN,+GA+30701"
1,"137 TAYLOR BRIDGE RD SW # A, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW # A,"CALHOUN, GA 30701",34.418477,-84.937531,,,,,,137+TAYLOR+BRIDGE+RD+SW+#+A,"CALHOUN,+GA+30701"
2,"137 TAYLOR BRIDGE RD SW, CALHOUN, GA 30701",137 TAYLOR BRIDGE RD SW,"CALHOUN, GA 30701",34.418477,-84.937531,,,,,,137+TAYLOR+BRIDGE+RD+SW,"CALHOUN,+GA+30701"
3,"314 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",314 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420551,-84.983000,,,,,,314+ROBBINS+CREEK+TRL+SW,"CALHOUN,+GA+30701"
4,"310 ROBBINS CREEK TRL SW, CALHOUN, GA 30701",310 ROBBINS CREEK TRL SW,"CALHOUN, GA 30701",34.420560,-84.983002,,,,,,310+ROBBINS+CREEK+TRL+SW,"CALHOUN,+GA+30701"


In [13]:

# OpenWeatherMap API Key

#api_key = api_keys.api_key

# Starting URL for Weather Map API Call
#url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key 

#url = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address=109 TAYLOR BRIDGE RD SW&citystatezip=CALHOUN GA 30701"  
#url = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address=5888+Colonnade+Drive&citystatezip=Rex,+GA+30273"  


In [24]:
base_url='http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address='

test=Data_set.head()

for index, row in test.iterrows():
    
    add=row['Zilliw format add']
    statezip=row['Zilliw format Zipcity']
    
    
    params=f'{add}&citystatezip={statezip}'
    
   
    response2 = requests.get(base_url, params=params)
    
    xmldoc = response2.text

109+TAYLOR+BRIDGE+RD+SW&citystatezip=CALHOUN,+GA+30701
137+TAYLOR+BRIDGE+RD+SW+#+A&citystatezip=CALHOUN,+GA+30701
137+TAYLOR+BRIDGE+RD+SW&citystatezip=CALHOUN,+GA+30701
314+ROBBINS+CREEK+TRL+SW&citystatezip=CALHOUN,+GA+30701
310+ROBBINS+CREEK+TRL+SW&citystatezip=CALHOUN,+GA+30701


In [20]:
xmldoc

'<?xml version="1.0" encoding="utf-8"?><SearchResults:searchresults xsi:schemaLocation="http://www.zillow.com/static/xsd/SearchResults.xsd https://www.zillowstatic.com/vstatic/18f7df5/static/xsd/SearchResults.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SearchResults="http://www.zillow.com/static/xsd/SearchResults.xsd"><request><address></address><citystatezip>CALHOUN, GA 30701</citystatezip></request><message><text>Error: no address specified</text><code>500</code></message></SearchResults:searchresults><!-- H:045  T:7ms  S:150  R:Mon Oct 01 12:48:09 PDT 2018  B:5.0.56469-master.7c8da2e~delivery_ready.08ee7f5 -->'

In [16]:
tree=ET.fromstring(xmldoc)
tree

<Element '{http://www.zillow.com/static/xsd/SearchResults.xsd}searchresults' at 0x113a81188>

In [17]:
results=tree.findall('./response/results/result')
results

[]

In [18]:
re=results[0]
dic=xmltodict.parse(ET.tostring(re))
#df=pd.DataFrame(dic['results'])
#zillow_data=pd.DataFrame(dic['result'])
#zipd=f'{zillow_data.iloc[0,0]}'
#zillow_data.iloc[0,-1]['#text']
#zestimate_dic=zillow_data.iloc[0,-2]
#zestimate=next(itertools.islice(zestimate_dic.values(), 1, 2)) 
dic['result']['lastSoldPrice']
dic['result']['zestimate']['amount']
dic['result']['totalRooms']

IndexError: list index out of range

In [ ]:
dic[]

In [ ]:
zid=ddd['amount']['zpid']
ddd